In [80]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import re
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [45]:
classes = os.listdir('./data/input_images')
classes

['paper', 'cardboard', 'plastic', 'metal', 'trash', 'glass']

In [46]:
def get_picture_per_classes(classes):
    return [os.listdir('./data/input_images/'+ item) for item in classes]

In [47]:
images = get_picture_per_classes(classes)

In [48]:
def get_image_name(item):
    return item.split('.')[0]

In [49]:
images_name = [get_image_name(item) for image in images for item in image]

In [50]:
def get_classes_from_images(item):
    num = re.findall('\d+', item)
    return item.split(num[0])[0]

In [51]:
image_classes = [get_classes_from_images(item) for item in images_name]

In [52]:
new_dict = {'Images_name':images_name, 'Image_class': image_classes}

In [53]:
p = pd.DataFrame(new_dict, index=images_name, columns=['Image_class'])
p.head()

,Image_class
paper85,paper
paper503,paper
paper75,paper
paper482,paper
paper300,paper


In [54]:
new_p = pd.get_dummies(p['Image_class'])
new_p.head()
new_p.to_csv(r'./images_dummies.csv')

In [85]:
# paths:
DATASET = 'images_dummies.csv'
ROOT_IMG = './data/input_images/'
ROOT_IMG_DAUGHTERS = classes
ROOT_MODELS = 'Models/'
# img size:
ROW_IMAGES = 384
COL_IMAGES = 512
IMG_COLORS = 3
input_shape = (ROW_IMAGES, COL_IMAGES, IMG_COLORS)
# num of pictures:
PIC_NUM = 2527
# train-test parameters:
TRAIN_RATIO = 0.2
RANDOM_STATE = 42
# fit parameters:
BATCH_SIZE = 20
EPOCHS =10

COLUMNS = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
num_classes = len(COLUMNS)

In [86]:
new_p['image'] = new_p.index

In [87]:
def get_image(item):
    return item + '.jpg'

In [88]:
def get_parental_path(item):
    parental = get_classes_from_images(item)
    return parental + '/'+ get_image(item)

In [89]:
def transform_path_to_image(item):
    imagen = cv2.imread(ROOT_IMG+get_parental_path(item))
    return imagen
    

In [93]:
def image_to_array(item):
    image = transform_path_to_image(item)
    # Converts image to an inx * iny size, change color to greyscale and flatten it into 1D ndarray
    image = cv2.resize(image,(256, 192)) 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = np.reshape(image,(256, 192))
    return np.ndarray.flatten(image)

In [94]:
new_p['image'] = new_p['image'].apply(image_to_array)
new_p.head()

,cardboard,glass,metal,paper,plastic,trash,image
paper85,0,0,0,1,0,0,"[240, 240, 239, 238, 237, 237, 236, 235, 234, ..."
paper503,0,0,0,1,0,0,"[208, 208, 208, 208, 208, 208, 207, 206, 205, ..."
paper75,0,0,0,1,0,0,"[223, 223, 223, 223, 222, 222, 222, 222, 222, ..."
paper482,0,0,0,1,0,0,"[238, 239, 238, 237, 236, 236, 235, 234, 234, ..."
paper300,0,0,0,1,0,0,"[217, 217, 217, 217, 216, 216, 216, 216, 216, ..."


def image_to_array(image,inx,iny):
    # Converts image to an inx * iny size, change color to greyscale and flatten it into 1D ndarray
    image = cv2.resize(image,(inx,iny)) 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = np.reshape(image,(inx,iny))
    return np.ndarray.flatten(image)

In [83]:
b = image_to_array(imagen, 256, 192)
b

array([235, 235, 233, ...,  75,  78,  78], dtype=uint8)

In [97]:
type(new_p.image[0])

numpy.ndarray

In [77]:
a = transform_path_to_image('paper/paper8.jpg')
image_to_array(a, 256, 192).shape

(49152,)

def transform_data(item):
    

COLUMNS = new_p.columns
data = pd.DataFrame(new_p,columns=['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash', 'image'])
data.reset_index(drop=True)
data.to_csv('data.csv')
data.head()

type(df.image[0])
imagen = cv2.imread(ROOT_IMG+'paper/paper1.jpg')
for lista in imagen:
    for pixel in lista:
        print(pixel/255)

def replace_value(item):
    return item.replace('\n', ',')

imagen= cv2.imread('./data/input_images/paper/paper7.jpg')
plt.imshow(Image.fromarray(imagen.squeeze()))


In [115]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(new_p.image), new_p.drop(columns=['image']), test_size=TRAIN_RATIO, random_state=RANDOM_STATE)

In [123]:
type(np.array(new_p.image))

numpy.ndarray

# Prepare data to feed the NN
num_classes = 6
# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, ROW_IMAGES/2, COL_IMAGES/2)
    X_test = X_test.reshape(X_test.shape[0], 1, ROW_IMAGES/2, COL_IMAGES/2)
    input_shape = (1, ROW_IMAGES/2, COL_IMAGES/2)
else:
    #X_train = X_train.reshape(X_train.shape[0], ROW_IMAGES/2, COL_IMAGES/2, 1)
    #X_test = X_test.reshape(X_test.shape[0], ROW_IMAGES/2, COL_IMAGES/2, 1)
    input_shape = (ROW_IMAGES/2, COL_IMAGES/2, 1)
# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [99]:
## This is the neural network proposed architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                activation='relu',
                input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])

batch_size = 20
epochs = 10
model.fit(X_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1,
         validation_data=(X_test, y_test))